In [ ]:
!pip install transformers -q

In [2]:
import os
os.chdir('/content/drive/MyDrive/doutorado/P_IA368DD_2023S/aula5')

In [ ]:
import json
from tqdm.auto import tqdm

import torch



from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
%%markdown
# Download and split MSMARCO Triples in Train-test datasets

# Download and split MSMARCO Triples in Train-test datasets


In [ ]:
#download datatset
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

In [ ]:
os.makedirs('data', exist_ok=True)
# Get the last 1000 for validation
!tail -n 1000 msmarco_triples.train.tiny.tsv > data/validation.tsv

# Get the first 10000 for training
!head -n 10000 msmarco_triples.train.tiny.tsv > data/train.tsv
     

In [4]:
%%markdown
#TREC-COVID 2020

#TREC-COVID 2020


In [ ]:
os.makedirs('TREC_COVID_2020', exist_ok=True)
# Corpus with all passages
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
!gzip -cd corpus.jsonl.gz > TREC_COVID_2020/corpus.jsonl


# Topics with all queries
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
!gzip -dc queries.jsonl.gz > TREC_COVID_2020/queries.jsonl


# Qrels with all relevances
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

In [ ]:
with open("TREC_COVID_2020/qrels.tsv", "w") as fout:
    with open("TREC_COVID_2020/test.tsv", "r") as fin:
        for idx, line in enumerate(fin):
            if idx != 0:
                qid, doc_id, relevance = line.strip().split("\t")
                fout.write(f"{qid}\t0\t{doc_id}\t{relevance}\n")

In [ ]:
os.makedirs('TREC_COVID_2020/corpus_original', exist_ok=True)

corpus = {}
with open("TREC_COVID_2020/corpus_original/corpus_original.jsonl", "w") as fout:
    with open("TREC_COVID_2020/corpus.jsonl", "r") as fin:
        for line in fin:
            doc = json.loads(line)
            corpus[doc["_id"]] = f"{doc['title']}. {doc['text']}"
            doc_dict = {
                "id": doc["_id"],
                "contents": f"{doc['title']}. {doc['text']}"
            }
            fout.write(json.dumps(doc_dict) + "\n")

In [6]:
%%markdown
# Start expanding corpus

# Start expanding corpus


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("results/final_model")
model = T5ForConditionalGeneration.from_pretrained("results/final_model").to(device)

In [ ]:
def get_batch(corpus, batch_size=256):
    ids = list(corpus.keys())
    texts = list(corpus.values())
    for i in tqdm(range(0, len(ids), batch_size), desc="Expanding documents"):
        yield ids[i:i+batch_size], texts[i:i+batch_size]

In [ ]:
os.makedirs('TREC_COVID_2020/expanded_corpus', exist_ok=True)

with open("TREC_COVID_2020/expanded_corpus/expanded_corpus.jsonl", "w") as fout:
    for ids, batch in get_batch(corpus, 256):
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)
        
        output_sequences = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            do_sample=False,
            max_new_tokens=32
        )
        new_queries = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
        for id, text, query in zip(ids, batch, new_queries):
            fout.write(json.dumps({
                "id": id,
                "contents": f"{query}. {text}"
            }) + "\n")
     